<a href="https://colab.research.google.com/github/rvargas42/Value_Growth_Challenge_ETSFactory/blob/main/data/extract_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Description
This notebook will take data from sources, filter it and create files to be used in model training

In [2]:
import requests as req
import pandas as pd
import os
import dotenv
import json
import pickle

import yfinance as yf
from investiny import historical_data
import investpy as inv

Load Directories and Files

In [3]:
data_dir = os.getcwd()
root_dir = os.path.dirname(data_dir)
dotenv_file = os.path.join(root_dir, '.env')

dotenv.load_dotenv()
FMP_KEY = os.getenv("FMP_KEY")
TE_KEY = os.getenv("TE_KEY")
FRED_KEY = os.getenv("FRED_KEY")

In [4]:
#FILES
all_data = os.listdir(data_dir)
etf_list = os.path.join(data_dir,"etf_data","fmp_data_etf_list.csv")
#READ DATA
etf_list = pd.read_csv(etf_list, index_col=False)
etf_list

,symbol,name,price,exchange,exchangeShortName,type
0,ULTR,IQ Ultra Short Duration ETF,47.96000,New York Stock Exchange Arca,AMEX,etf
1,RXL,ProShares Ultra Health Care,102.61300,New York Stock Exchange Arca,AMEX,etf
2,FMNY,First Trust New York High Income Municipal ETF,26.66290,New York Stock Exchange Arca,AMEX,etf
3,UIMP.DE,UBS (Lux) Fund Solutions – MSCI USA Socially R...,196.32000,Frankfurt Stock Exchange,XETRA,etf
4,DANC.TO,Desjardins Alt Long/Short Equity Market Neutra...,22.26000,Toronto Stock Exchange,TSX,etf
...,...,...,...,...,...,...
11965,TQSM.TO,TD Q U.S. Small-Mid-Cap Equity ETF,22.43000,Toronto Stock Exchange,TSX,etf
11966,SSIL.L,WisdomTree Silver 1x Daily Short,6.45625,London Stock Exchange,LSE,etf
11967,GURU,Global X Guru Index ETF,42.95410,New York Stock Exchange Arca,AMEX,etf
11968,HMJI.TO,BetaPro Marijuana Companies Inverse ETF,23.61000,Toronto Stock Exchange,TSX,etf


In [5]:
etf_list.dropna(inplace=True)
etfs_us = etf_list[etf_list["exchangeShortName"].str.contains("AMEX")]
etfs_eu = etf_list[etf_list["name"].str.contains("Europe")]
etfs_eu

,symbol,name,price,exchange,exchangeShortName,type
12,VERG.L,Vanguard FTSE Developed Europe ex UK UCITS ETF,39.52768,London Stock Exchange,LSE,etf
35,S6EW.L,Ossiam STOXX Europe 600 Equal Weight NR UCITS ETF,125.44000,London Stock Exchange,LSE,etf
46,INSU.SW,Lyxor Index Fund - Lyxor STOXX Europe 600 Insu...,73.40000,Swiss Exchange,SIX,etf
49,3165.HK,ChinaAMC MSCI Europe Quality Hedged to USD ETF,18.82000,HKSE,HKSE,etf
85,DXSQ.DE,Xtrackers II iTraxx Europe Swap UCITS ETF,119.70500,Frankfurt Stock Exchange,XETRA,etf
...,...,...,...,...,...,...
11892,CS9.PA,Amundi Index Solutions - Amundi MSCI Europe Ex...,329.48800,Paris,EURONEXT,etf
11895,UTIL.L,SSgA SPDR ETFs Europe II Public Limited Compan...,164.70000,London Stock Exchange,LSE,etf
11896,DXSC.DE,XtrackersMSCI Europe Materials ESG Screened UC...,164.86000,Frankfurt Stock Exchange,XETRA,etf
11903,EEUE.PA,BNPP E MSCI Europe ESG Filtered Min TE,15.86800,Paris,EURONEXT,etf


In [6]:
etfs_us_growth = etfs_us[etfs_us["name"].str.contains("Growth", na=False)]
etfs_us_value = etfs_us[etfs_us["name"].str.contains("Value",na=False)]
etfs_growth_value = {
    "Growth": {i : {"description":"","price_history":[], "sector_exposure":[]} for i in etfs_us_growth.symbol}, 
    "Value": {i : {"description":"","price_history":[], "sector_exposure":[]} for i in etfs_us_value.symbol}
}
print(f"-> {len(etfs_growth_value["Growth"].keys())} US Growth ETFs")
print(f"-> {len(etfs_growth_value["Value"].keys())} US Value ETFs")
etfs_us_value

-> 94 US Growth ETFs
-> 102 US Value ETFs


,symbol,name,price,exchange,exchangeShortName,type
24,RZV,Invesco S&P SmallCap 600 Pure Value ETF,102.2810,New York Stock Exchange Arca,AMEX,etf
57,IWN,iShares Russell 2000 Value ETF,151.8200,New York Stock Exchange Arca,AMEX,etf
60,IVLU,iShares Edge MSCI Intl Value Factor ETF,28.7600,New York Stock Exchange Arca,AMEX,etf
83,JVAL,JPMorgan U.S. Value Factor ETF,40.5600,New York Stock Exchange Arca,AMEX,etf
140,JPSV,Jpmorgan Active Small Cap Value ETF,54.0500,New York Stock Exchange Arca,AMEX,etf
...,...,...,...,...,...,...
11363,IVE,iShares S&P 500 Value ETF,183.3000,New York Stock Exchange Arca,AMEX,etf
11397,GVLU,Gotham 1000 Value ETF,23.1700,New York Stock Exchange Arca,AMEX,etf
11497,IHYV,Invesco Corporate Income Value ETF,24.8350,New York Stock Exchange Arca,AMEX,etf
11591,VOOV,Vanguard S&P 500 Value Index Fund,177.1700,New York Stock Exchange Arca,AMEX,etf


In [8]:
etfs_eu_value = etfs_eu[etfs_eu["name"].str.contains("Value", na=False)]
etfs_eu_value

,symbol,name,price,exchange,exchangeShortName,type
144,CEMS.DE,iShares Edge MSCI Europe Value Factor UCITS ETF,8.7130,Frankfurt Stock Exchange,XETRA,etf
220,CAPE.PA,Ossiam Shiller Barclays Cape Europe Sector Val...,517.4000,Paris,EURONEXT,etf
288,SGASF,SSgA SPDR ETFs Europe II Public Limited Compan...,56.7750,Other OTC,PNK,etf
426,EUPE.DE,Ossiam Shiller Barclays Cape Europe Sector Val...,517.4000,Frankfurt Stock Exchange,XETRA,etf
714,EL4D.DE,Deka STOXX Europe Strong Value 20 UCITS ETF,24.4400,Frankfurt Stock Exchange,XETRA,etf
834,LCPE.L,Ossiam Shiller Barclays Cape Europe Sector Val...,43905.0000,London Stock Exchange,LSE,etf
1623,EMSV.DE,Invesco MSCI Europe Value UCITS ETF,262.1500,Frankfurt Stock Exchange,XETRA,etf
1806,EVAL.L,SPDR MSCI Europe Value UCITS ETF,41.7700,London Stock Exchange,LSE,etf
1904,EMSV.F,Invesco MSCI Europe Value UCITS ETF,261.5000,Frankfurt Stock Exchange,XETRA,etf
3612,CAPE.L,Ossiam Shiller Barclays Cape Europe Sector Val...,525.7000,London Stock Exchange,LSE,etf


#### ETF Data

In [10]:
reference_etfs = ["SPYV", "SPYG", "RZV"]
european_etfs = ["CEMS.DE", "VALU.DE"]

for e in reference_etfs:
    data = yf.Ticker(e).history(period='max', interval="1d").to_period("d")["Close"]
    data.to_csv(os.path.join(data_dir, "etf_data", f"{e}.csv"))

for e in european_etfs:
    data = yf.Ticker(e).history(period='max', interval="1d").to_period("d")["Close"]
    data.to_csv(os.path.join(data_dir, "etf_data", f"{e}.csv"))

C:\Users\rvarg\AppData\Local\Temp\ipykernel_13736\3500583058.py:5: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  data = yf.Ticker(e).history(period='max', interval="1d").to_period("d")["Close"]
C:\Users\rvarg\AppData\Local\Temp\ipykernel_13736\3500583058.py:5: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  data = yf.Ticker(e).history(period='max', interval="1d").to_period("d")["Close"]
C:\Users\rvarg\AppData\Local\Temp\ipykernel_13736\3500583058.py:5: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  data = yf.Ticker(e).history(period='max', interval="1d").to_period("d")["Close"]
C:\Users\rvarg\AppData\Local\Temp\ipykernel_13736\3500583058.py:9: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  data = yf.Ticker(e).history(period='max', interval="1d").to_period("d")["Close"]
C:\Users\rvarg\AppData\Local\Tem

#### OTHER ASSETS

In [28]:
currency_tickers = ["EURUSD=X"]
futures = ["GC=F", "CL=F", "HO=F", "ZB=F", "ZC=F"]

for c in currency_tickers:
    data = yf.Ticker(c).history(period='max', interval="1d").to_period("d")["Close"]
    data.to_csv(os.path.join(data_dir, "currencies", f"{c.replace("=X","")}.csv"))

for t in futures:
    data = yf.Ticker(t).history(period='max', interval="1d").to_period("d")["Close"]
    data.to_csv(os.path.join(data_dir, "commodities", f"{t.replace("=F","_F")}.csv"))

C:\Users\rvarg\AppData\Local\Temp\ipykernel_12540\3674600216.py:5: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  data = yf.Ticker(c).history(period='max', interval="1d").to_period("d")["Close"]
C:\Users\rvarg\AppData\Local\Temp\ipykernel_12540\3674600216.py:9: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  data = yf.Ticker(t).history(period='max', interval="1d").to_period("d")["Close"]
C:\Users\rvarg\AppData\Local\Temp\ipykernel_12540\3674600216.py:9: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  data = yf.Ticker(t).history(period='max', interval="1d").to_period("d")["Close"]
C:\Users\rvarg\AppData\Local\Temp\ipykernel_12540\3674600216.py:9: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  data = yf.Ticker(t).history(period='max', interval="1d").to_period("d")["Close"]
C:\Users\rvarg\AppData\Local\Tem

### Macro Data

In [25]:
from datetime import datetime
from fredapi import Fred

macro_data = {}
fred = Fred(api_key=FRED_KEY)
macro_series = ["FEDFUNDS"]
for series in macro_series:
    data = fred.get_series(series)
    data.columns = macro_series
    data.to_csv(os.path.join(data_dir, "macro_data", f"{series}.csv"))
